In [64]:
from dotenv import load_dotenv
from openai import OpenAI
from io import BytesIO
import requests
import json
import os

load_dotenv()
#키 가져오기
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [65]:
#객체 생성
client = OpenAI()

In [66]:
#Responses API를 사용하여 파일 검색을 수행하기 전에, 
#벡터 스토어에 지식 기반을 설정하고 해당 스토어에 파일을 업로드
def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id



In [67]:
file_id = create_file(client, './howto-sockets.pdf')

file-UXq4aEbqf5Y72ySkNgVzyV


In [68]:
#vector store 생성. 여러 벡터 저장소 생성 가능
vector_store = client.vector_stores.create(
    name='knowledge_base'
)

print(vector_store.id)

vs_685cd263b1cc8191b83aea4d339d813c


In [69]:
#파일을 vector store에 추가
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)


VectorStoreFile(id='file-UXq4aEbqf5Y72ySkNgVzyV', created_at=1750913636, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_685cd263b1cc8191b83aea4d339d813c', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [70]:
result_list = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)

print(result_list)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-UXq4aEbqf5Y72ySkNgVzyV', created_at=1750913636, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_685cd263b1cc8191b83aea4d339d813c', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-UXq4aEbqf5Y72ySkNgVzyV', last_id='file-UXq4aEbqf5Y72ySkNgVzyV')


In [71]:
#지식 기반이 설정되면, 
#모델에 사용할 수 있는 도구 목록에 file_search 도구를 포함
#검색할 벡터저장소 목록도 함께 포함시킬 수 있음
response = client.responses.create(
    model='gpt-4.1',
    input='파이썬 코드로 소켓 만드는 방법을 간단하게 설명해줘',
    tools=[{
        "type" : "file_search",
        "vector_store_ids" : [vector_store.id]
    }],
    tool_choice='required'
)

print(response)

Response(id='resp_685cd264e6fc8199af240ab38f76c5b101978c5833a00910', created_at=1750913636.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_685cd265a400819993dc27cbd0d729e601978c5833a00910', queries=['파이썬 소켓 만드는 방법 간단 설명', '파이썬 소켓 예제 코드', 'socket 모듈 사용법', '파이썬 서버 소켓', '파이썬 클라이언트 소켓'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_685cd269df8c8199898c8547e9ed862201978c5833a00910', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-UXq4aEbqf5Y72ySkNgVzyV', filename='howto-sockets.pdf', index=1333, type='file_citation'), AnnotationFileCitation(file_id='file-UXq4aEbqf5Y72ySkNgVzyV', filename='howto-sockets.pdf', index=1333, type='file_citation')], text='파이썬에서 소켓을 만드는 방법은 간단합니다. 기본적으로 `socket` 모듈을 사용합니다. 간단한 TCP 클라이언트와 서버 예제도 함께 설명드리겠습니다.\n\n---\n\n### 1. 소켓 만들기\n\n먼저 소켓 객체를 생성합니다.\n```python\nimport socket\ns 

In [74]:
print(dict(response))

{'id': 'resp_685cd264e6fc8199af240ab38f76c5b101978c5833a00910', 'created_at': 1750913636.0, 'error': None, 'incomplete_details': None, 'instructions': None, 'metadata': {}, 'model': 'gpt-4.1-2025-04-14', 'object': 'response', 'output': [ResponseFileSearchToolCall(id='fs_685cd265a400819993dc27cbd0d729e601978c5833a00910', queries=['파이썬 소켓 만드는 방법 간단 설명', '파이썬 소켓 예제 코드', 'socket 모듈 사용법', '파이썬 서버 소켓', '파이썬 클라이언트 소켓'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_685cd269df8c8199898c8547e9ed862201978c5833a00910', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-UXq4aEbqf5Y72ySkNgVzyV', filename='howto-sockets.pdf', index=1333, type='file_citation'), AnnotationFileCitation(file_id='file-UXq4aEbqf5Y72ySkNgVzyV', filename='howto-sockets.pdf', index=1333, type='file_citation')], text='파이썬에서 소켓을 만드는 방법은 간단합니다. 기본적으로 `socket` 모듈을 사용합니다. 간단한 TCP 클라이언트와 서버 예제도 함께 설명드리겠습니다.\n\n---\n\n### 1. 소켓 만들기\n\n먼저 소켓 객체를 생성합니다.\n```python

In [75]:
print(response.output_text)

파이썬에서 소켓을 만드는 방법은 간단합니다. 기본적으로 `socket` 모듈을 사용합니다. 간단한 TCP 클라이언트와 서버 예제도 함께 설명드리겠습니다.

---

### 1. 소켓 만들기

먼저 소켓 객체를 생성합니다.
```python
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
```
- `AF_INET`: IPv4(인터넷 주소 패밀리)
- `SOCK_STREAM`: TCP 소켓

---

### 2. TCP 클라이언트 예제

서버에 접속하고 메시지를 보내는 간단한 클라이언트 예시입니다.
```python
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("서버주소", 12345))  # ("hostname", port)
s.sendall(b"Hello, server!")
data = s.recv(1024)  # 최대 1024바이트 받기
print("Received", repr(data))
s.close()
```

---

### 3. TCP 서버 예제

클라이언트의 접속을 기다리고 메시지를 받는 서버 예시입니다.
```python
import socket

serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serversocket.bind(("0.0.0.0", 12345))  # 모든 네트워크 인터페이스에서 수신
serversocket.listen(1)  # 동시에 1개의 연결을 기다림

while True:
    clientsocket, address = serversocket.accept()
    print("Connection from", address)
    data = clientsocket.recv(1024)
    clientsocket.sendall(b"Hello, client!")
    client

In [ ]:
#pip install streamlit

  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------  10.0/10.1 MB 61.8 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 36.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 29.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 6.9/6.9 MB 42.6 MB/s eta 0:00:00
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ----------------- ---------------------- 11.3/25.8 MB 54.2 MB/s eta 0:00:01
   ------------------------------------ --- 23.6/25.8 MB 57.5 MB/s eta 0:00:01
   ---------------------------------------- 25.8/25.8 MB 49.6 MB/s eta 0:00:00

   ----------------------------------------  0/20 [wat

In [ ]:
#pip install beautifulsoup4

  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)

   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   ---------------------------------------- 2/2 [beautifulsoup4]

Note: you may need to restart the kernel to use updated packages.
